# Hearthstone Deck Recommendation
### Objetivo: Construir un mazo ganador en contra de un mazo oponente específico para una partida de Hearthstone.

## Librerias

In [6]:
import pandas as pd
import numpy as np
from collections import Counter
import keras
import pickle

## Base de datos
La base de datos se obtuvo de la plataforma HearthPwn (https://www.hearthpwn.com/decks), esta contiene información sobre mazos creados por jugadores de Hearthstone. Los datos incluyen la clase del mazo, sus 30 cartas, el coste de creación, la puntuación, entre otros.

In [7]:
df = pd.read_csv("data\data.csv")
df.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


In [8]:
df.describe()

,craft_cost,deck_id,rating,card_0,card_1,card_2,card_3,card_4,card_5,card_6,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
count,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,...,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000
mean,5645.360218,394152.425798,2.687230,166.555443,215.682814,328.068948,388.254399,465.439497,530.422084,618.649576,...,6676.817657,7879.285156,9310.508977,10780.480210,12401.890674,13709.898080,15068.109406,15955.823881,19239.277147,20537.999847
std,3927.986295,222605.617140,22.117751,515.546751,549.163776,609.109069,629.218897,710.534945,813.475129,987.250288,...,12610.593768,13750.165455,14885.066464,15862.422929,16731.209629,17295.441879,17766.351821,18039.395857,18567.676290,18686.462303
min,0.000000,18.000000,0.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
25%,2720.000000,216721.500000,1.000000,64.000000,75.000000,189.000000,237.000000,279.000000,304.000000,401.000000,...,1158.000000,1363.000000,1659.000000,1783.000000,1794.000000,1913.000000,2010.000000,2037.000000,2078.000000,2095.000000
50%,5000.000000,406046.500000,1.000000,138.000000,180.000000,285.000000,315.000000,415.000000,475.000000,559.000000,...,1940.000000,2029.000000,2061.000000,2078.000000,2275.000000,2488.000000,2577.000000,2682.000000,2901.000000,3015.000000
75%,7740.000000,590820.500000,1.000000,238.000000,285.000000,421.000000,476.000000,605.000000,643.000000,763.000000,...,2610.000000,2736.000000,2890.000000,2958.000000,38391.000000,38526.000000,38727.000000,38833.000000,38918.000000,39034.000000
max,48000.000000,749548.000000,4016.000000,41409.000000,41409.000000,41609.000000,41409.000000,41409.000000,41609.000000,41609.000000,...,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41841.000000,42146.000000,42146.000000,42146.000000


In [4]:
df['deck_class'].value_counts()

deck_class
Mage       45306
Priest     44307
Paladin    42266
Warlock    38022
Druid      37891
Shaman     36457
Warrior    35944
Rogue      34794
Hunter     31245
Name: count, dtype: int64

La clase de mazo con más entradas es "Mage". Para la implementación del sistema de recomendación utilizaremos solo mazos mago.

In [5]:
df = df[df['deck_class'] == 'Mage']

## Cartas y mazos
Definimos el conjunto *cards* con las *N* cartas más repetidas en los mazos mago y *D* como el tamaño del mazo para armar.

In [12]:
cards_cols = df.iloc[:, 11:41] # columnas con las cartas en la base de datos
all_cards = (cards_cols.values.ravel()).tolist() # lista con las cartas
counter = Counter(all_cards) # contador de las cartas

N = 200 # tamaño conjunto cartas para elegir
cards =  [item[0] for item in counter.most_common(N)] # conjunto con las N cartas más repetidas

D = 30 # tamaño mazo Hearthstone

El mazo lo definimos como un vector binario *deck* con *N* entradas y exactamente *D* iguales a 1. Este vector nos indica con la i-esima entrada si está o no (1 o 0) en el mazo la i-esima carta de *cards*. Definimos la función *binary_to_cards* que transforma el vector *deck* en una lista con la enumeración de las cartas del mazo.

Para la construción de un mazo ganador necesitaremos el mazo inicial *deck_p* del jugador, que tenemos el objetivo de mejorar, y el mazo del oponente *deck_o*, que buscamos derrotar en una partida. A continación se presenta un ejemplo de mazos.

In [13]:
def binary_to_cards(deck, cards):
    return [int(x) for x in deck*cards if x != 0]

In [14]:
deck_p = np.zeros(N, dtype=int) # definir mazo jugador
deck_p[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio jugador
print(f'Vector mazo jugador: {deck_p}')
print(f'Cartas mazo jugador: {binary_to_cards(deck_p, cards)}')
print()

deck_o = np.zeros(N, dtype=int) # definir mazo oponente
deck_o[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio oponente
print(f'Vector mazo oponente: {deck_o}')
print(f'Cartas mazo oponente: {binary_to_cards(deck_o, cards)}')

Vector mazo jugador: [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
Cartas mazo jugador: [2064, 1659, 904, 778, 1783, 757, 1117, 1362, 2409, 1092, 985, 1037, 75, 272, 192, 30, 587, 890, 2408, 2736, 609, 654, 1100, 281, 754, 1029, 503, 1022, 999, 1371]

Vector mazo oponente: [0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 

## Transiciones y acciones
El proceso para mejorar el mazo *deck_p* consta de aplicar múltiples veces la función de transición *transition* determinada por una acción. Una acción *action* es una tupla con las cartas del reemplazo que modifica exactamente una carta del mazo por una carta que actualmente no está incluida. A continuación, un ejemplo del uso de la función *transition*, en donde se elige de forma aleatoria la acción.

In [15]:
def transition(deck, action):
    deck[action[0]] = 0 # eliminamos la carta antigua del mazo
    deck[action[1]] = 1 # agregamos la carta nueva al mazo
    return deck

In [16]:
print(f'Mazo antiguo: {deck_p}')

old_card = np.random.choice(np.where(deck_p == 1)[0]) # elegimos aleatoriamente una carta que está en el mazo
new_card = np.random.choice(np.where(deck_p == 0)[0]) # elegimos aleatoriamente una carta que no está en el mazo
action = (old_card, new_card) # definimos la acción
print(f'Acción: ({action[0]},{action[1]})')

new_deck_p = transition(deck_p, action)
print(f'Mazo nuevo: {new_deck_p}')
print(f'Cartas mazo jugador: {binary_to_cards(new_deck_p, cards)}')

Mazo antiguo: [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
Acción: (129,150)
Mazo nuevo: [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
Cartas mazo jugador: [2064, 1659, 904, 778, 1783, 757, 1117, 1362, 2409, 1092, 985, 1037, 75, 272, 192, 30, 587, 890, 2736, 609, 654, 1100, 42

## Win Rate
Para elegir las acciones que nos lleven a encontrar un mazo ganador usaremos el *win_rate* de este mazo. Entonces, para elegir la próxima acción buscaremos cual es la que maximiza el *win_rate* del mazo.

Para hacer una función predictora del *win_rate* entrenamos distintos modelos de machine learning, esto se realizó en el notebook *win_rate_prediction.ipynb*.

A continuación, se presenta la función *win_rate_NN* y un ejemplo de su uso con un mazo.

In [17]:
model = keras.models.load_model("win_rate_NN.keras")
model.summary()
with open("encoder", "rb") as f:
    encoder = pickle.load(f)
with open("win_rate_RF.pkl", "rb") as f:
    rf = pickle.load(f)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout                │ (None, 32)             │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         1,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_dropout_1              │ (None, 50)             │             0 │
│ (GaussianDropout)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,883 (54.23 KB)

 Trainable params: 4,627 (18.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,256 (36.16 KB)

In [ ]:
def win_rate_NN(deck,encoder = encoder, model =model):
  mazo_gen_num = pd.DataFrame([{f"card_{i}": (deck[i] if i>-1 else 5645) for i in range(-1,30)}]).rename(columns={"cards_-1":"craft_cost"})
  hero = "Mage"
  mazo_gen_cat = pd.DataFrame([{"hero":hero}])
  encoded_categorical = encoder.transform(mazo_gen_cat)
  # # Step 3: Convert the encoded data back to a DataFrame
  encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
  # # Step 4: Concatenate the numerical column with the one-hot encoded categorical columns
  final_df = pd.concat([mazo_gen_num, encoded_df], axis=1).rename(columns={"craft_cost":"dust"})
  results = model.predict(final_df)
  return results

In [35]:
deck_o_cards = binary_to_cards(new_deck_p, cards)
print(win_rate_NN(deck_o_cards))

   card_-1  card_0  card_1  card_2  card_3  card_4  card_5  card_6  card_7  \
0     5645    2064    1659     904     778    1783     757    1117    1362   

   card_8  ...  card_20  card_21  card_22  card_23  card_24  card_25  card_26  \
0    2409  ...      654     1100      420      281      754     1029      503   

   card_27  card_28  card_29  
0     1022      999     1371  

[1 rows x 31 columns]
   card_-1  card_0  card_1  card_2  card_3  card_4  card_5  card_6  card_7  \
0     5645    2064    1659     904     778    1783     757    1117    1362   

   card_8  ...  card_29  hero_Druid  hero_Hunter  hero_Mage  hero_Paladin  \
0    2409  ...     1371         0.0          0.0        1.0           0.0   

   hero_Priest  hero_Rogue  hero_Shaman  hero_Warlock  hero_Warrior  
0          0.0         0.0          0.0           0.0           0.0  

[1 rows x 40 columns]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
[[54.60872]]


Aquí se puede ver el win rate aproximado obtenido a partir del mazo generado para el oponente